<a href="https://colab.research.google.com/github/ReginaLin2005/skin-cancer-detector/blob/skin-cancer-detection-model/Skin_Cancer_Detection_Model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.models import load_model

In [3]:
X = []
y = []

benign_path = '/content/drive/MyDrive/dataset/benign'
malignant_path = '/content/drive/MyDrive/dataset/malignant'
malignant_path_2 = '/content/drive/MyDrive/skin_cancer_dataset'

In [4]:
# Function to load images from a directory
def load_images_from_folder(folder, label):
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Ensure it's an image
            img_path = os.path.join(folder, filename)
            try:
                img = Image.open(img_path).convert('RGB')  # Convert image to RGB
                img = img.resize((224, 224))  # Resize to match input size of the model
                img = np.array(img)  # Convert image to NumPy array
                X.append(img)
                y.append(label)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")

In [5]:
# Load benign images (label 0)
load_images_from_folder(benign_path, 0)

# Load malignant images (label 1)
load_images_from_folder(malignant_path, 1)
#load_images_from_folder(malignant_path_2, 1)

# Convert lists to NumPy arrays for use in ML models
X = np.array(X)
y = np.array(y)

In [6]:
X

array([[[[234, 154, 147],
         [235, 156, 149],
         [237, 160, 150],
         ...,
         [229, 163, 137],
         [230, 158, 136],
         [230, 158, 143]],

        [[233, 155, 145],
         [233, 159, 148],
         [236, 160, 146],
         ...,
         [230, 163, 146],
         [228, 163, 143],
         [228, 160, 137]],

        [[234, 158, 145],
         [233, 155, 143],
         [238, 156, 145],
         ...,
         [227, 161, 139],
         [228, 162, 140],
         [223, 157, 133]],

        ...,

        [[223, 155, 136],
         [222, 152, 127],
         [223, 151, 126],
         ...,
         [214, 143, 121],
         [215, 141, 128],
         [215, 146, 131]],

        [[220, 152, 129],
         [223, 152, 132],
         [219, 149, 124],
         ...,
         [213, 138, 119],
         [217, 145, 130],
         [215, 146, 130]],

        [[223, 152, 134],
         [223, 152, 132],
         [223, 148, 127],
         ...,
         [218, 143, 124],
        

In [7]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Adding Batch Normalization
    tf.keras.layers.BatchNormalization(),

    # Optional additional Conv2D layer
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Adding Dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification output
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Resize and normalize the images
X_train_resized = [tf.image.resize(image, (150, 150)) for image in X_train]
X_train_resized = np.array(X_train_resized) / 255.0  # Normalize pixel values to [0, 1]

# Fit the model
history = model.fit(
    X_train_resized,
    y_train,
    epochs=5
)

Epoch 1/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 353s 4s/step - accuracy: 0.6938 - loss: 0.5821
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 373s 4s/step - accuracy: 0.7971 - loss: 0.4233
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 383s 4s/step - accuracy: 0.8020 - loss: 0.4011
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 377s 4s/step - accuracy: 0.7907 - loss: 0.4262
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 380s 4s/step - accuracy: 0.8256 - loss: 0.3716


In [11]:
# Resize validation images to 150x150 and normalize pixel values
X_val_resized = np.array([img_to_array(array_to_img(img, scale=False).resize((150, 150))) for img in X_val])
X_val_resized = X_val_resized / 255.0  # Normalize the images to [0, 1] range

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_val_resized, y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


22/22 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7170 - loss: 0.5977
Validation Loss: 0.5992485880851746
Validation Accuracy: 0.7163742780685425


In [12]:
# Save the trained model to a file
model.save('cancer_detection_model.h5')

In [13]:
# Load the saved model
loaded_model = load_model('cancer_detection_model.h5')

In [14]:
def preprocess_image(image_path):
    # Load the image and resize it to (150, 150) to match the model's input shape
    img = Image.open(image_path)
    img = img.resize((150, 150))  # Resize to 150x150
    img_array = np.array(img)     # Convert to NumPy array
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension
    return img_array

In [15]:
def predict_image(model, image_path):
    # Preprocess the input image
    processed_image = preprocess_image(image_path)

    # Make a prediction using the model
    prediction = loaded_model.predict(processed_image)

    # Since you're doing binary classification, the output will be a probability
    # You can set a threshold (e.g., 0.5) to classify benign (0) or malignant (1)
    if prediction[0] > 0.5:
        return "Malignant"
    else:
        return "Benign"